In [1]:
import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv("train_with_embeddings.csv")

In [3]:
# new features
train_data["url_count"] = train_data["urls"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["text_len"] = train_data["text"].apply(lambda s: len(s))
train_data["hashtags_count"] = train_data["hashtags"].apply(lambda s: s[1:-1].count("\'")/2)
train_data["day"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).day)
train_data["hour"] = train_data["timestamp"].apply(lambda t: datetime.utcfromtimestamp(t/1000).hour)

# indicators of keywords
train_data["Macron"] =  train_data["text"].apply(lambda s: ("macron" in s.lower().split()))
train_data["Zemmour"] =  train_data["text"].apply(lambda s: ("zemmour" in s.lower().split()))
train_data["Melenchon"] =  train_data["text"].apply(lambda s: ("melenchon" in s.replace("é","e").lower().split()))
train_data["rt"] =  train_data["text"].apply(lambda s: ("rt" in s.lower().split()))


In [4]:
# TODO: include sent analysis

In [5]:
# select useful columns
train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)
# train_data_filtered = train_data.loc[:, ["retweets_count","favorites_count","followers_count","statuses_count","friends_count",
#                                  "hashtags_count","hour","verified","url_count","text_len","rt","Macron","Zemmour","Melenchon"]]


X_train, X_eval, y_train, y_eval = train_test_split(train_data_filtered.drop("retweets_count", axis=1),
                                                    train_data_filtered["retweets_count"],
                                                    random_state=42, test_size=0.1)

# Standardize the data
normal_columns = train_data_filtered.drop(["hour", "verified", "Macron", "Zemmour", "Melenchon", "url_count", "rt", "retweets_count"], axis=1).columns
mu, sigma = X_train[normal_columns].mean(axis=0), X_train[normal_columns].std(axis=0)
X_train.loc[:, normal_columns] = (X_train[normal_columns] - mu) / sigma
X_eval.loc[:, normal_columns] = (X_eval[normal_columns] - mu) / sigma

In [6]:
import torch
import pytorch_lightning as pl
from setup_data import TweetsDataset,TweetsDatasetEmbedded
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader, random_split
from tweet_pred_model import TweetPredModel

device = torch.device("cpu")

In [7]:
# Importing model
model = TweetPredModel(embed_len=20, data_len=train_data_filtered.shape[1]-20, output_channel=8, dense_size=20)


In [8]:
dataset = TweetsDatasetEmbedded(train_data_filtered, 20)

# Split into training and test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = random_split(dataset, [train_size, test_size])

# # Dataloaders
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=12)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=12)

# train(train_data_filtered, 3)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=50)
trainer.fit(model=model, train_dataloaders=trainloader, val_dataloaders=testloader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/users/eleves-b/2020/bruno.rabelo-wenchenck-botelho/challenge/env/lib64/python3.6/site-packages/pytorch_lightning/trainer/trainer.py:1585: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."

  | Name    | Type   | Params
-----------------------------------
0 | conv1   | Conv1d | 1.3 K 
1 | linear1 | Linear | 272   
2 | conv2   | Conv1d | 520   
3 | linear2 | Linear | 72    
4 | linear3 | Linear | 500   
5 | linear4 | Linear | 420   
6 | linear5 | Linear | 21    
-----------------------------------
3.1 K     Trainable params
0         Non-trainable params
3.1 K     Total params
0.012     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

UnboundLocalError: local variable 'x1' referenced before assignment